Consider the n-dimensional linear system of ordinary differential equations:
$$x'(t)=A(t)x(t)$$
where in addition, the matrix $A(t)$ is periodic with period $T$, i.e. $A(t+T)=A(t)$ for all real values of $t$.
Let $Φ(t)$ denote the principal fundamental matrix solution such that the columns
of $Φ(t)$ are linearly independent, and $Φ(0)=I$. Then, Floquet's theorem decomposes the principal
fundamental matrix as the product of a periodic matrix $P(t)$ with period $T$ and an exponential matrix $e^{tB}$.



In [1]:
using Plots, ApproxFun

In [2]:
function Base.expm{T<:Number}(A::Matrix{T},f::Fun)
    D,V = eig(A)
    fC = D == real(D) ? fill(f,n) : fill(f+0im,n)
    for i = 1:length(D)
        fC[i] = Fun(t->exp(D[i]*fC[i](t)),domain(f))
    end
    V*diagm(fC)*inv(V)
end;

Consider the system of coupled harmonic oscillators with periodic parameterized excitation:
     $$x'' + (1+a \cos(2t))x = x-y$$
     $$y'' + (1+a \cos(2t))y = y-x$$
We calculate the Principal Fundamental Matrix $Φ$.

In [3]:
T = π;a=0.15
t = Fun(identity,0..T)
d=domain(t)
D=Derivative(d)
B=ldirichlet(d)
n=4
A=[ diagm(fill(ldirichlet(d),n));
    D             -I  0I            0I;
   (2+a*cos(2t))   D  -I            0I;
   0I             0I   D            -I;
   -I             0I  (2+a*cos(2t))  D]

# we need eye(n) for the initial condition and zeros(n,n) for the
# right-hand side of the ODE

Φ = A\eye(2n,n);  

It can be further expressed as $Φ(t) = P(t) e^{tB}$ where $P(t)$ is a periodic matrix, and $B$ is constant.
Using the values of $Φ(T)$:

In [4]:
ΦT = Φ(T)

4×4 Array{Float64,2}:
 -0.170879  -0.148885  -0.836059   0.265569
  0.732284  -0.170879  -0.612945  -0.836059
 -0.836059   0.265569  -0.170879  -0.148885
 -0.612945  -0.836059   0.732284  -0.170879

we calculate the matrix $B$ and the Floquet exponents and multipliers.

In [5]:
B = logm(ΦT)/T
D,V = eig(B)

println("Floquet Exponents = \n")
[println(D[i]) for i=1:n]
println("\nFloquet Multipliers = \n")
[println(exp(D[i]*T)) for i=1:n];

Floquet Exponents = 

-1.6653345369377348e-16 - 0.26835469053190525im
-8.630249292984615e-17 + 0.2683546905319051im
0.03747531973228935 + 0.9999999999999997im
-0.037475319732288806 + 1.0im

Floquet Multipliers = 

0.6651802570066604 - 0.7466828146465888im
0.665180257006661 + 0.7466828146465889im
-1.1249427991479373 + 1.1369156774202889e-15im
-0.8889340869219566 + 1.0886302842019492e-16im


We can build the matrix exponential $e^{-tB}$ and use it to find the periodic matrix $P(t)$.
Periodicity is numerically confirmed by `Fun` instantiation with low number of coefficients.

In [6]:
PI = Φ*expm(B,-t)
P = Fun(t->PI(t),PeriodicInterval(d))
P|>ncoefficients

223

With the matrix B and the periodic matrix P(t), we can construct 
the solution with any initial conditions for as long as we want!

In [7]:
t = Fun(identity,0..10T)
d = domain(t)
x0 = rand(n)

PI = Fun(t->P(t),d)
xsol = real(PI*expm(B,t)*x0)

plot(mat(xsol))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 596.063,223.074 596.058,223.077 596.049,223.082 596.035,223.089 596.017,223.099 595.994,223.111 595.966,223.125 595.933,223.142 595.896,223.161 595.855,223.182 
 595.808,223.205 595.757,223.23 595.702,223.257 595.642,223.286 595.577,223.317 595.507,223.349 595.433,223.384 595.355,223.419 595.271,223.456 595.183,223.495 
 595.091,223.534 594.994,223.575 594.892,223.617 594.785,223.659 594.674,223.703 594.559,223.747 594.439,223.792 594.314,223.837 594.184,223.883 594.05,223.929 
 593.912,223.976 593.768,224.023 593.621,224.07 593.468,224.117 593.311,224.165 593.15,224.213 592.983,224.261 592.813,224.31 592.637,224.359 592.458,224.408 
 592.273,224.458 592.084,224.508 591.89,224.559 591.692,224.611 591.489,224.665 591.282,224.719 591.07,224.776 590.854,224.834 590.633,224.894 590.408,224.957 
 590.178,225.023 589.943,225.092 589.704,225.165 589.46,225.241 589.212,225.323 588.959,225.41 588.702,225.502 588.441,225.601 588.174,225.707 587.904,225.821 
 587.629,225.943 587.349,226.074 587.065,226.215 586.776,226.367 586.483,226.53 586.185,226.705 585.883,226.894 585.577,227.097 585.266,227.315 584.95,227.549 
 584.631,227.799 584.306,228.068 583.977,228.355 583.644,228.663 583.307,228.991 582.965,229.34 582.618,229.713 582.267,230.109 581.912,230.529 581.552,230.975 
 581.188,231.448 580.82,231.947 580.447,232.474 580.07,233.03 579.688,233.615 579.302,234.23 578.912,234.875 578.518,235.551 578.119,236.258 577.715,236.997 
 577.308,237.767 576.896,238.568 576.48,239.401 576.059,240.265 575.634,241.16 575.205,242.084 574.772,243.039 574.334,244.021 573.892,245.032 573.446,246.068 
 572.995,247.13 572.54,248.214 572.081,249.32 571.618,250.444 571.151,251.586 570.679,252.741 570.203,253.908 569.723,255.083 569.239,256.263 568.75,257.444 
 568.258,258.622 567.761,259.794 567.26,260.954 566.755,262.098 566.246,263.221 565.732,264.318 565.215,265.383 564.693,266.411 564.167,267.394 563.637,268.328 
 563.104,269.205 562.565,270.018 562.023,270.761 561.477,271.426 560.927,272.006 560.373,272.493 559.814,272.88 559.252,273.158 558.686,273.32 558.115,273.358 
 557.541,273.263 556.963,273.029 556.38,272.646 555.794,272.108 555.204,271.407 554.609,270.535 554.011,269.486 553.409,268.252 552.803,266.827 552.193,265.206 
 551.579,263.383 550.962,261.354 550.34,259.113 549.714,256.658 549.085,253.986 548.452,251.096 547.815,247.985 547.174,244.655 546.529,241.106 545.881,237.341 
 545.228,233.362 544.572,229.174 543.913,224.782 543.249,220.193 542.582,215.415 541.91,210.457 541.236,205.329 540.557,200.043 539.875,194.611 539.189,189.048 
 538.499,183.367 537.806,177.586 537.109,171.722 536.408,165.792 535.704,159.815 534.996,153.812 534.284,147.802 533.569,141.806 532.851,135.847 532.128,129.945 
 531.402,124.124 530.673,118.404 529.94,112.81 529.203,107.363 528.463,102.084 527.72,96.9968 526.973,92.1214 526.222,87.479 525.468,83.0897 524.711,78.9729 
 523.95,75.1473 523.186,71.6303 522.418,68.4388 521.647,65.5882 520.872,63.0929 520.094,60.9662 519.313,59.22 518.529,57.8648 517.741,56.9098 516.949,56.3629 
 516.155,56.2303 515.357,56.5169 514.556,57.226 513.751,58.3591 512.943,59.9165 512.132,61.8965 511.318,64.2959 510.501,67.1097 509.68,70.3312 508.856,73.9518 
 508.029,77.9613 507.199,82.3474 506.366,87.0962 505.529,92.1916 504.689,97.616 503.847,103.349 503.001,109.371 502.152,115.656 501.3,122.18 500.445,128.915 
 499.587,135.834 498.726,142.906 497.862,150.098 496.994,157.378 496.124,164.712 495.251,172.064 494.375,179.399 493.496,186.679 492.614,193.869 491.73,200.932 
 490.842,207.832 489.951,214.534 489.058,221.002 488.162,227.205 487.262,233.111 486.361,238.69 485.456,243.916 484.548,248.765 483.638,253.215 482.725,257.249 
 481.809,260.851 480.89,264.011 479.969,26